## Make predictions

This notebook is designed to generate heat maps on TSP20, TSP50 and TSP1000. Now let's make some predictions on the test dataset with the trained model. 

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
import json
import argparse
import time
import math
import numpy as np
from scipy.special import softmax
import torch
import torch.nn.functional as F
import torch.nn as nn

from utils.plot_utils import plot_predictions_cluster
from utils.test_utils import test_one_tsp, multiprocess_write
from config import *
from sklearn.utils.class_weight import compute_class_weight

# Remove warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from scipy.sparse import SparseEfficiencyWarning
warnings.simplefilter('ignore', SparseEfficiencyWarning)

from utils.process import *
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

from data.data_generator import tsp_instance_reader

from utils.tsplib import read_tsplib_coor, read_tsplib_opt, write_tsplib_prob


from multiprocessing import Pool
from multiprocessing import cpu_count

import tqdm

In [3]:
torch.cuda.is_available()

True

### 1. Loading trained Att-GCN based on TSP20-trainset

In [4]:
# model-parameter
config_path = "./configs/tsp20.json"
config = get_config(config_path)

# setting random seed to 1
if torch.cuda.is_available():
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed_all(1)
    print("Using CUDA!")
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(1)

Using CUDA!


In [5]:
# Instantiate the network
net = nn.DataParallel(ResidualGatedGCNModel(config, dtypeFloat, dtypeLong))
if torch.cuda.is_available():
    net.cuda()  
# Define optimizer
learning_rate = config.learning_rate
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
# Load checkpoint
log_dir = f"./logs/{config.expt_name}/"
if torch.cuda.is_available():
    # TSP-20
    checkpoint = torch.load("./logs/tsp20/best_val_checkpoint.tar")
else:
    checkpoint = torch.load("./logs/tsp20/best_val_checkpoint.tar", map_location='cpu')
# Load network state
net.load_state_dict(checkpoint['model_state_dict'])
# Load optimizer state
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# Load other training parameters
epoch = checkpoint['epoch']
train_loss = checkpoint['train_loss']
val_loss = checkpoint['val_loss']
for param_group in optimizer.param_groups:
    learning_rate = param_group['lr']  

### 2. Graph Sampling 

In [6]:
net.eval()

DataParallel(
  (module): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=2, out_features=300, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=150, bias=False)
    (edges_embedding): Embedding(3, 150)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Conv1d(300, 300, kernel_size=(1,), stride=(1,))
          (V): Conv1d(300, 300, kernel_size=(1,), stride=(1,))
        )
        (edge_feat): EdgeFeatures(
          (U): Conv1d(300, 300, kernel_size=(1, 1), stride=(1,))
          (V): Conv1d(300, 300, kernel_size=(1,), stride=(1,))
        )
        (bn_node): BatchNormNode(
          (batch_norm): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
        (bn_edge): BatchNormEdge(
          (batch_norm): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        )
      )
      (1): ResidualGatedGCNLaye

#### 2.1 Heatmap generator on TSP20

All output would be stored on the dir `./results/heatmap/tsp20`. After running the next node code cell, we would get 10,000 probabilistic heat maps for TSP20-instances and then copy them to the dir `MCTS/tsp-20-50-100/heatmap`.

In [7]:
num_nodes = 20
f = open('./data/tsp{}_test_concorde.txt'.format(num_nodes), 'r')
testset_tsp = f.readlines()
f.close()

config.expt_name = 'tsp{}'.format(num_nodes)
K = 19
avg_mean_rank = [] 
top_k, cluster_center = K, 0
batch_size = 250 
if num_nodes == 20:
    threshold = 1
else:
    threshold = math.ceil((num_nodes / (top_k+1) ) * 5)
epoch = int(len(testset_tsp)/batch_size)
buff_coor = np.zeros(shape=(num_nodes, 2), dtype = np.float64)
start_row_num = 0

In [8]:
# init
count_buff = np.zeros(shape=(batch_size*threshold, ), dtype=np.int32)
edges = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.int32)
edges_values = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.float16)
nodes = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
nodes_coord = np.zeros(shape = (batch_size*threshold, K+1, 2), dtype=np.float16)
edges_target = np.zeros(shape = (batch_size*threshold, K+1, K+1), dtype=np.int32)
nodes_target = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
meshs = np.zeros(shape = (batch_size*threshold, 2, K+1, K+1), dtype=np.int32)
Omegas = np.zeros(shape = (batch_size, num_nodes, num_nodes), dtype=np.int32)
opts = np.zeros(shape = (batch_size, num_nodes+1), dtype=np.int32)
num_neighbors = config.num_neighbors
beam_size = config.beam_size

#start = time.time()
sum_time = 0
for j in tqdm.tqdm(range(epoch)):
    start = time.time()
    for i in range(batch_size):
        edge, edges_value, node, node_coord, edge_target, node_target, mesh, omega, opt = test_one_tsp(tsp_source=testset_tsp[start_row_num+i], 
                                                                                      coor_buff=buff_coor, node_num=num_nodes, 
                                                                                      cluster_center=0, top_k=K)
        edges[i*threshold:(i+1)*threshold, ...] = edge
        edges_values[i*threshold:(i+1)*threshold, ...] = edges_value
        nodes[i*threshold:(i+1)*threshold, ...] = node
        nodes_coord[i*threshold:(i+1)*threshold, ...] = node_coord
        edges_target[i*threshold:(i+1)*threshold, ...] = edge_target
        nodes_target[i*threshold:(i+1)*threshold, ...] = node_target
        meshs[i*threshold:(i+1)*threshold, ...] = mesh
        Omegas[i, ...] = omega
        opts[i, ...] = opt


    with torch.no_grad():
        # Convert batch to torch Variables
        x_edges = Variable(torch.LongTensor(edges).type(dtypeLong), requires_grad=False)
        x_edges_values = Variable(torch.FloatTensor(edges_values).type(dtypeFloat), requires_grad=False)
        x_nodes = Variable(torch.LongTensor(nodes).type(dtypeLong), requires_grad=False)
        x_nodes_coord = Variable(torch.FloatTensor(nodes_coord).type(dtypeFloat), requires_grad=False)
        y_edges = Variable(torch.LongTensor(edges_target).type(dtypeLong), requires_grad=False)
        y_nodes = Variable(torch.LongTensor(nodes_target).type(dtypeLong), requires_grad=False)

        # Compute class weights
        edge_labels = y_edges.cpu().numpy().flatten()
        edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)

        # Forward pass
        y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
        y_preds_prob = F.softmax(y_preds, dim=3)
        y_preds_prob_numpy = y_preds_prob.cpu().numpy()

    # multi - processes
#     progress_pool = Pool(processes=10)
#     for i in range(batch_size):
#         heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
#         progress_pool.apply_async(multiprocess_write, args=(y_preds_prob_numpy[i*thre:(i+1)*thre, ...],
#                                                            meshs[i*thre:(i+1)*thre, ...], Omegas[i, ...],
#                                                            num_nodes, heatmap_path, True, opts[i, ...]))
#     progress_pool.close()
#     progress_pool.join()
    end = time.time()
    sum_time += end - start
    # single - process
    for i in range(batch_size):
        heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
        rank = multiprocess_write(y_preds_prob_numpy[i*threshold:(i+1)*threshold, ...],
                                                           meshs[i*threshold:(i+1)*threshold, ...], Omegas[i, ...],
                                                           num_nodes, heatmap_path, True, opts[i, ...])
        avg_mean_rank.append(rank)
    start_row_num+= batch_size


  0%|          | 0/40 [00:00<?, ?it/s]/home/FINDER-TSP/attgcn_preprocessor/utils/test_utils.py:86: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  edges_value = distB_raw[mesh].copy()
/home/FINDER-TSP/attgcn_preprocessor/utils/test_utils.py:90: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Omega[mesh] += 1
/home/FINDER-TSP/attgcn_preprocessor/utils/test_utils.py:112: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an

#### 2.2 Heatmap generator on TSP50

All output would be stored on the dir `./results/heatmap/tsp50`. After running the next node code cell, we would get 10,000 probabilistic heat maps for TSP50-instances and then copy them to the dir `MCTS/tsp-20-50-100/heatmap`.

In [ ]:
num_nodes = 50
f = open('./data/tsp{}_test_concorde.txt'.format(num_nodes), 'r')
testset_tsp = f.readlines()
#testset_tsp = testset_tsp[:100]
f.close()

config.expt_name = 'tsp{}'.format(num_nodes)
K = 19
avg_mean_rank = [] 
top_k, cluster_center = K, 0
batch_size = 250 
threshold = math.ceil((num_nodes / (top_k+1) ) * 5)
epoch = int(len(testset_tsp)/batch_size)
buff_coor = np.zeros(shape=(num_nodes, 2), dtype = np.float64)
start_row_num = 0

In [ ]:
# init
K_expand = 29
count_buff = np.zeros(shape=(batch_size*threshold, ), dtype=np.int32)
edges = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.int32)
edges_values = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.float16)
nodes = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
nodes_coord = np.zeros(shape = (batch_size*threshold, K+1, 2), dtype=np.float16)
edges_target = np.zeros(shape = (batch_size*threshold, K+1, K+1), dtype=np.int32)
nodes_target = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
meshs = np.zeros(shape = (batch_size*threshold, 2, K+1, K+1), dtype=np.int32)
Omegas = np.zeros(shape = (batch_size, num_nodes, num_nodes), dtype=np.int32)
opts = np.zeros(shape = (batch_size, num_nodes+1), dtype=np.int32)
num_neighbors = config.num_neighbors
beam_size = config.beam_size

sum_time = 0
for j in tqdm.tqdm(range(epoch)):
    start = time.time()
    for i in range(batch_size):
        edge, edges_value, node, node_coord, edge_target, node_target, mesh, omega, opt = test_one_tsp(tsp_source=testset_tsp[start_row_num+i], 
                                                                                      coor_buff=buff_coor, node_num=num_nodes, 
                                                                                      cluster_center=0, top_k=K, top_k_expand=K_expand)
        edges[i*threshold:(i+1)*threshold, ...] = edge
        edges_values[i*threshold:(i+1)*threshold, ...] = edges_value
        nodes[i*threshold:(i+1)*threshold, ...] = node
        nodes_coord[i*threshold:(i+1)*threshold, ...] = node_coord
        edges_target[i*threshold:(i+1)*threshold, ...] = edge_target
        nodes_target[i*threshold:(i+1)*threshold, ...] = node_target
        meshs[i*threshold:(i+1)*threshold, ...] = mesh
        Omegas[i, ...] = omega
        opts[i, ...] = opt


    with torch.no_grad():
        # Convert batch to torch Variables
        x_edges = Variable(torch.LongTensor(edges).type(dtypeLong), requires_grad=False)
        x_edges_values = Variable(torch.FloatTensor(edges_values).type(dtypeFloat), requires_grad=False)
        x_nodes = Variable(torch.LongTensor(nodes).type(dtypeLong), requires_grad=False)
        x_nodes_coord = Variable(torch.FloatTensor(nodes_coord).type(dtypeFloat), requires_grad=False)
        y_edges = Variable(torch.LongTensor(edges_target).type(dtypeLong), requires_grad=False)
        y_nodes = Variable(torch.LongTensor(nodes_target).type(dtypeLong), requires_grad=False)

        # Compute class weights
        edge_labels = y_edges.cpu().numpy().flatten()
        edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)

        # Forward pass
        y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
        y_preds_prob = F.softmax(y_preds, dim=3)
        y_preds_prob_numpy = y_preds_prob.cpu().numpy()

    # multi - processes
#     progress_pool = Pool(processes=10)
#     for i in range(batch_size):
#         heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
#         progress_pool.apply_async(multiprocess_write, args=(y_preds_prob_numpy[i*thre:(i+1)*thre, ...],
#                                                            meshs[i*thre:(i+1)*thre, ...], Omegas[i, ...],
#                                                            num_nodes, heatmap_path, True, opts[i, ...]))
#     progress_pool.close()
#     progress_pool.join()
    end = time.time()
    sum_time += end - start
    # single - process
    for i in range(batch_size):
        heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
        rank = multiprocess_write(y_preds_prob_numpy[i*threshold:(i+1)*threshold, ...],
                                                           meshs[i*threshold:(i+1)*threshold, ...], Omegas[i, ...],
                                                           num_nodes, heatmap_path, True, opts[i, ...])
        avg_mean_rank.append(rank)
    start_row_num+= batch_size


#### 2.3 Heatmap generator on TSP100

All output would be stored on the dir `./results/heatmap/tsp100`. After running the next node code cell, we would get 10,000 probabilistic heat maps for TSP100-instances and then copy them to the dir `MCTS/tsp-20-50-100/heatmap`.

In [ ]:
num_nodes = 100
f = open('./data/tsp{}_test_concorde.txt'.format(num_nodes), 'r')
testset_tsp = f.readlines()
f.close()

config.expt_name = 'tsp{}'.format(num_nodes)
K = 19
avg_mean_rank = [] 
top_k, cluster_center = K, 0
batch_size = 250 
threshold = math.ceil((num_nodes / (top_k+1) ) * 5)
epoch = int(len(testset_tsp)/batch_size)
buff_coor = np.zeros(shape=(num_nodes, 2), dtype = np.float64)
start_row_num = 0

In [ ]:
# init
K_expand = 39
count_buff = np.zeros(shape=(batch_size*threshold, ), dtype=np.int32)
edges = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.int32)
edges_values = np.zeros(shape=(batch_size*threshold, K+1, K+1), dtype=np.float16)
nodes = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
nodes_coord = np.zeros(shape = (batch_size*threshold, K+1, 2), dtype=np.float16)
edges_target = np.zeros(shape = (batch_size*threshold, K+1, K+1), dtype=np.int32)
nodes_target = np.zeros(shape = (batch_size*threshold, K+1), dtype=np.int32)
meshs = np.zeros(shape = (batch_size*threshold, 2, K+1, K+1), dtype=np.int32)
Omegas = np.zeros(shape = (batch_size, num_nodes, num_nodes), dtype=np.int32)
opts = np.zeros(shape = (batch_size, num_nodes+1), dtype=np.int32)
num_neighbors = config.num_neighbors
beam_size = config.beam_size

sum_time = 0
for j in tqdm.tqdm(range(epoch)):
    start = time.time()
    for i in range(batch_size):
        edge, edges_value, node, node_coord, edge_target, node_target, mesh, omega, opt = test_one_tsp(tsp_source=testset_tsp[start_row_num+i], 
                                                                                      coor_buff=buff_coor, node_num=num_nodes, 
                                                                                      cluster_center=0, top_k=K, top_k_expand=K_expand)
        edges[i*threshold:(i+1)*threshold, ...] = edge
        edges_values[i*threshold:(i+1)*threshold, ...] = edges_value
        nodes[i*threshold:(i+1)*threshold, ...] = node
        nodes_coord[i*threshold:(i+1)*threshold, ...] = node_coord
        edges_target[i*threshold:(i+1)*threshold, ...] = edge_target
        nodes_target[i*threshold:(i+1)*threshold, ...] = node_target
        meshs[i*threshold:(i+1)*threshold, ...] = mesh
        Omegas[i, ...] = omega
        opts[i, ...] = opt


    with torch.no_grad():
        # Convert batch to torch Variables
        x_edges = Variable(torch.LongTensor(edges).type(dtypeLong), requires_grad=False)
        x_edges_values = Variable(torch.FloatTensor(edges_values).type(dtypeFloat), requires_grad=False)
        x_nodes = Variable(torch.LongTensor(nodes).type(dtypeLong), requires_grad=False)
        x_nodes_coord = Variable(torch.FloatTensor(nodes_coord).type(dtypeFloat), requires_grad=False)
        y_edges = Variable(torch.LongTensor(edges_target).type(dtypeLong), requires_grad=False)
        y_nodes = Variable(torch.LongTensor(nodes_target).type(dtypeLong), requires_grad=False)

        # Compute class weights
        edge_labels = y_edges.cpu().numpy().flatten()
        edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)

        # Forward pass
        y_preds, loss = net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
        y_preds_prob = F.softmax(y_preds, dim=3)
        y_preds_prob_numpy = y_preds_prob.cpu().numpy()

    # multi - processes
#     progress_pool = Pool(processes=10)
#     for i in range(batch_size):
#         heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
#         progress_pool.apply_async(multiprocess_write, args=(y_preds_prob_numpy[i*thre:(i+1)*thre, ...],
#                                                            meshs[i*thre:(i+1)*thre, ...], Omegas[i, ...],
#                                                            num_nodes, heatmap_path, True, opts[i, ...]))
#     progress_pool.close()
#     progress_pool.join()
    end = time.time()
    sum_time += end - start
    # single - process
    for i in range(batch_size):
        heatmap_path = f'./results/heatmap/tsp{num_nodes}/heatmaptsp{num_nodes}_{i+start_row_num}.txt'
        rank = multiprocess_write(y_preds_prob_numpy[i*threshold:(i+1)*threshold, ...],
                                                           meshs[i*threshold:(i+1)*threshold, ...], Omegas[i, ...],
                                                           num_nodes, heatmap_path, True, opts[i, ...])
        avg_mean_rank.append(rank)
    start_row_num+= batch_size
